<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Mixed_Integer_Problem_for_Draft_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1  Initialisierung und Konfiguration

## 1.1 Bibliotheken installieren und laden


In [5]:
try:
    import mip
except ImportError:
    import sys
    !{sys.executable} -m pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

## 1.2 Liga-Parameter und Rahmenbedingungen definieren

In [6]:
year = 2024
num_teams = 12
num_rounds = 15
num_weeks = 17
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'K': 1, 'DST': 1}

# 2 Datenbeschaffung und -vorverarbeitung


## 2.1 ADP, Bye Weeks und Projections laden

In [16]:
adp_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
player_adp = (
    pd.read_csv(adp_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'adp': 'ADP', 'points': 'TTL'})
      .fillna({'ADP': 999})
      .loc[:, ['Player', 'POS', 'ADP', 'TTL']]
      .sort_values('ADP')
      .reset_index(drop=True)
)


bye_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/bye_weeks/bye_weeks_{year}.csv'
bye_weeks = pd.read_csv(bye_url)


season_projection_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'
season_projections = (
    pd.read_csv(season_projection_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'team': 'Team', 'points': 'TTL', 'adp': 'ADP'})
      .loc[:, ['Player', 'POS', 'Team', 'TTL', 'ADP']]
      .merge(bye_weeks[['Abbreviation', 'Bye']], left_on='Team', right_on='Abbreviation', how='left')
      .drop(columns='Abbreviation')
      .drop_duplicates(subset='Player', keep='first')
)
season_projections['avg_proj'] = season_projections['TTL'] / np.where(
    season_projections['Bye'].isna(),
    num_weeks,
    num_weeks - 1
)
week_cols = [f'Week_{w}' for w in range(1, num_weeks + 1)]
for week in week_cols:
    season_projections[week] = np.where(
        season_projections['Bye'] == week,
        0,
        season_projections['avg_proj']
    )
season_projections = season_projections.drop(columns='avg_proj')

## 2.4 Berechnung von TTL, Dropoff und VOR

In [21]:
# Gewichte
dropoff_w = {'QB':1.0,'RB':1.0,'WR':1.0,'TE':0.9,'K':0.4,'DST':0.3}
vor_w     = {'QB':0.8,'RB':1.0,'WR':1.0,'TE':0.8,'K':0.25,'DST':0.25}

# Replacement-TTL je Position ermitteln (wie zuvor berechnet)
rep_ttl = (
    season_projections
      .groupby('POS')['TTL']
      .apply(lambda s: s.nlargest(lineup_req[s.name]*num_teams)
                   .iloc[-1]
             if len(s) >= lineup_req[s.name]*num_teams else 0)
)

# Komplettes Pipeline-Update inkl. ADP-Fillna
season_projections = (
    season_projections
      .sort_values(['POS','TTL'], ascending=[True,False])
      .assign(
          dropoff=lambda df: (
              df.groupby('POS')['TTL']
                .diff(-1)
                .fillna(0)
                .mul(df['POS'].map(dropoff_w))
          ),
          VOR=lambda df: (
              (df['TTL'] - df['POS'].map(rep_ttl))
                .clip(lower=0)
                .mul(df['POS'].map(vor_w))
          )
      )
      .fillna({'dropoff': 0, 'VOR': 0, 'ADP': 999})
      .sort_values(['ADP', 'TTL'], ascending=[True, False])
      .reset_index(drop=True)
)


In [22]:
season_projections

,Player,POS,Team,TTL,ADP,Bye,Week_1,Week_2,Week_3,Week_4,...,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,dropoff,VOR
0,Christian McCaffrey,RB,SF,353.0,1.36,Week_9,22.062500,22.062500,22.062500,22.062500,...,22.062500,22.062500,22.062500,22.062500,22.062500,22.062500,22.062500,22.062500,44.00,162.0
1,CeeDee Lamb,WR,DAL,348.0,3.88,Week_7,21.750000,21.750000,21.750000,21.750000,...,21.750000,21.750000,21.750000,21.750000,21.750000,21.750000,21.750000,21.750000,14.00,119.0
2,Tyreek Hill,WR,MIA,334.0,4.02,Week_6,20.875000,20.875000,20.875000,20.875000,...,20.875000,20.875000,20.875000,20.875000,20.875000,20.875000,20.875000,20.875000,21.00,105.0
3,Breece Hall,RB,NYJ,309.0,4.98,Week_12,19.312500,19.312500,19.312500,19.312500,...,19.312500,19.312500,0.000000,19.312500,19.312500,19.312500,19.312500,19.312500,6.00,118.0
4,Bijan Robinson,RB,ATL,303.0,5.19,Week_12,18.937500,18.937500,18.937500,18.937500,...,18.937500,18.937500,0.000000,18.937500,18.937500,18.937500,18.937500,18.937500,27.00,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,Giants,DST,NYG,85.5,185.00,Week_11,5.343750,5.343750,5.343750,5.343750,...,5.343750,0.000000,5.343750,5.343750,5.343750,5.343750,5.343750,5.343750,0.06,0.0
275,John Stephens Jr.,WR,DAL,145.0,999.00,Week_7,9.062500,9.062500,9.062500,9.062500,...,9.062500,9.062500,9.062500,9.062500,9.062500,9.062500,9.062500,9.062500,0.00,0.0
276,Darren Waller,TE,FA,106.0,999.00,NaN,6.235294,6.235294,6.235294,6.235294,...,6.235294,6.235294,6.235294,6.235294,6.235294,6.235294,6.235294,6.235294,4.50,0.0
277,Kareem Hunt,RB,FA,103.0,999.00,NaN,6.058824,6.058824,6.058824,6.058824,...,6.058824,6.058824,6.058824,6.058824,6.058824,6.058824,6.058824,6.058824,1.00,0.0


# 3 Modellparameter und Variablenaufbau

## 3.1 Spielerlisten und Parameter-Mapping

## 3.2 Optimierungsvariablen und Zielfunktion konfigurieren

# 4 Entwurfsphase (Draft-Optimierung)

## 4.1 Draft-Reihenfolge generieren

## 4.2 Optimierungsmodell für den MIP-Draft

## 4.3 Gegnerische Picks simulieren

# 5 Ergebnisaufbereitung

## 5.1 Draft-Log exportieren und zusammenführen

## 5.2 Ergebnis-Statustext ausgeben

# 6 Evaluation